# Writing ROS JTC Action Client

Combining our ROS knowledge with the documentation on the JTC (Joint Trajectory Controller) from the [ROS website](https://control.ros.org/humble/doc/ros2_controllers/joint_trajectory_controller/doc/userdoc.html#using-joint-trajectory-controller-s), we can set up an action client that communicates with the action server in our robot. For this to work, we have to make sure that our ROS UR Driver is running in its own kernel.

We can do that with this command:

In [ ]:
ros2 launch ur_robot_driver ur_control.launch.py ur_type:=ur3e robot_ip:=130.64.17.5 launch_rviz:=false

After that we can write a minimal ROS Action Client for the JTC. This will allow us to send actions to the JTC that contain joint angles for the robot arm and also monitor the progress of those actions.

In [ ]:
# ROS Objects
import rclpy
from rclpy.node import Node
from rclpy.action import ActionClient

# ROS Message Types
from std_msgs.msg import String
from builtin_interfaces.msg import Duration
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from control_msgs.action import FollowJointTrajectory


class RobotActionClient(Node):

    def __init__(self):
        super().__init__('RobotClientNode')

        # Action Client Setup
        controller_name = (
            "scaled_joint_trajectory_controller/follow_joint_trajectory"
        )
        self.joints = [
            "shoulder_pan_joint",
            "shoulder_lift_joint",
            "elbow_joint",
            "wrist_1_joint",
            "wrist_2_joint",
            "wrist_3_joint",
        ]
        self._action_client = ActionClient(self, FollowJointTrajectory, controller_name)

        # Callback futures
        self._send_goal_future = None
        self._get_result_future = None

        # State of action
        self.done = True

    # Wrapper function to send trajectory
    def send_action(self, joint_positions):
        self.done = False
        trajectory = self.make_trajectory(joint_positions)
        self.execute_trajectory(trajectory)

    # Package trajectory points and times into JointTrajectory object
    def make_trajectory(self, joint_positions):
        trajectory = JointTrajectory()
        trajectory.joint_names = self.joints

        for i,position in enumerate(joint_positions):
            point = JointTrajectoryPoint()
            point.positions = position # radians
            point.time_from_start = Duration(sec=3*(i+1),nanosec=0) # 3 sec per position
            trajectory.points.append(point)

        return trajectory

    # Send action containing trajectory
    def execute_trajectory(self,trajectory):
        goal = FollowJointTrajectory.Goal()
        goal.trajectory = trajectory
        self._action_client.wait_for_server()
        self._send_goal_future = self._action_client.send_goal_async(goal)
        self._send_goal_future.add_done_callback(self.goal_response_callback)

    # Callback for trajectory acceptance or rejection
    def goal_response_callback(self,future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().info(f"Goal #{self._id} rejected :(")
            self.done = True
        else:
            self.get_logger().debug(f"Goal #{self._id} accepted :)")
            self._get_result_future = goal_handle.get_result_async()
            self._get_result_future.add_done_callback(self.get_result_callback)

    # Callback for completion of trajectory
    def get_result_callback(self,future):
        result = future.result().result
        if result.error_code == FollowJointTrajectory.Result.SUCCESSFUL:
            self.get_logger().info("Goal Completed")
            self.done = True
        else:
            self.get_logger().info("Something went wrong with the goal")
            self.done = True

def main():
    rclpy.init()

    robot = RobotActionClient()

    joint_positions = [[0.305, -1.766, 1.763, -1.569, -1.573, -1.268]] # radians

    try:
        robot.send_action(joint_positions)
        while not robot.done:
            # stay here until action is completed or fails
            rclpy.spin_once(robot)
    except:
        robot.destroy_node()
        rclpy.shutdown()
        print('Done')

if __name__ == '__main__':
    main()